In [1]:
import pandas as pd
!pip install dash -qq
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
nfl_salary_raw = pd.read_excel('NFL SALARY.xlsx')
nfl_raw = pd.read_csv('maddennfl24fullplayerratings.csv')
nba_raw = pd.read_csv('current_nba_players.csv')
fifa_raw = pd.read_csv('male_players.csv')

In [17]:
def feet_inches_to_cm(height_str):
    try:
        if pd.isna(height_str):
            return None
        height_str = str(height_str).replace(' ', '').replace('"', '')
        parts = height_str.split("'")
        feet = float(parts[0])
        inches = float(parts[1]) if len(parts) > 1 and parts[1] else 0
        total_inches = feet * 12 + inches
        cm = total_inches * 2.54
        return round(cm)
    except:
        return None

In [18]:
def lbs_to_kg(weight_lbs):
    if pd.isna(weight_lbs):
            return None
    weight = pd.to_numeric(weight_lbs, errors='coerce')
    if pd.isna(weight):
            return None
    weight_kg = weight * 0.45359237
    return round(weight_kg)

In [28]:
def inches_to_cm(Height):
    if pd.isna(Height):
            return None
    Height = pd.to_numeric(Height, errors='coerce')
    if pd.isna(Height):
            return None
    height_cm = Height * 2.54
    return round(height_cm)

In [38]:
nfl = nfl_raw[['Full Name', 'Overall Rating', 'Height','Years Pro', 'Weight', 'Age']]
nfl = nfl.merge(
    nfl_salary_raw,
    left_on='Full Name',
    right_on='Player',
    how='inner')

if not pd.api.types.is_numeric_dtype(nfl['APY']):
    nfl['APY'] = nfl['APY'].replace('[\$,]', '', regex=True).astype(float)
nfl.sort_values('APY', ascending=False, inplace=True)
nfl.drop_duplicates('Full Name', keep='first', inplace=True)
nfl=nfl[['Full Name', 'Overall Rating', 'Height', 'Weight', 'Age','Years Pro', 'Pos.', 'APY']]
nfl

,Full Name,Overall Rating,Height,Weight,Age,Years Pro,Pos.,APY
752,Dak Prescott,87,74,238,30,7,QB,60000000.0
589,Joe Burrow,95,76,215,26,3,QB,55000000.0
1488,Jordan Love,70,76,219,24,3,QB,55000000.0
249,Trevor Lawrence,82,78,220,23,2,QB,55000000.0
16,Tua Tagovailoa,83,73,217,25,3,QB,53100000.0
...,...,...,...,...,...,...,...,...
885,Ian Book,55,72,206,25,2,QB,225000.0
819,Ochaun Mathis,64,76,247,24,0,EDGE,225000.0
1433,Kyle Philips,71,71,189,24,1,WR,225000.0
769,Charles Harris,71,75,250,28,6,EDGE,225000.0


In [39]:
nfl['height_cm'] = nfl['Height'].apply(inches_to_cm)
nfl['weight_kg'] = nfl['Weight'].apply(lbs_to_kg)
nfl.drop(['Height', 'Weight'], axis=1, inplace=True)
nfl.rename(columns={'Full Name': 'name', 'Overall Rating': 'overall', 'Years Pro': 'years_pro', 'Pos.': 'position'}, inplace=True)
nfl= nfl[['name', 'overall', 'height_cm', 'weight_kg', 'Age', 'years_pro', 'position', 'APY']]
nfl

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY
752,Dak Prescott,87,188,108,30,7,QB,60000000.0
589,Joe Burrow,95,193,98,26,3,QB,55000000.0
1488,Jordan Love,70,193,99,24,3,QB,55000000.0
249,Trevor Lawrence,82,198,100,23,2,QB,55000000.0
16,Tua Tagovailoa,83,185,98,25,3,QB,53100000.0
...,...,...,...,...,...,...,...,...
885,Ian Book,55,183,93,25,2,QB,225000.0
819,Ochaun Mathis,64,193,112,24,0,EDGE,225000.0
1433,Kyle Philips,71,180,86,24,1,WR,225000.0
769,Charles Harris,71,190,113,28,6,EDGE,225000.0


In [48]:
target_date = pd.to_datetime('2024-08-30')
nba_raw['Age'] = pd.to_datetime(nba_raw['birthdate'], errors='coerce').apply(
    lambda x: relativedelta(target_date, x).years if pd.notnull(x) else None)
nba = nba_raw[['name', 'overall','height_feet', 'weight_lbs', 'Age','years_in_the_nba', 'position_1', 'season_salary']]
nba

,name,overall,height_feet,weight_lbs,Age,years_in_the_nba,position_1,season_salary
0,Aaron Gordon,85,"6'8""",235,28,11.0,PF,22841455.0
1,Aaron Holiday,74,"6'0""",185,27,7.0,PG,4668000.0
2,Aaron Nesmith,79,"6'5""",215,24,5.0,SF,11000000.0
3,Aaron Wiggins,79,"6'5""",190,25,4.0,PF,10514017.0
4,Adam Flagler,67,"6'3""",180,24,2.0,SG,NaN
...,...,...,...,...,...,...,...,...
521,Zach Edey,79,"7'5""",305,22,1.0,C,5756880.0
522,Zach LaVine,85,"6'5""",200,29,11.0,SG,43031940.0
523,Zeke Nnaji,72,"6'9""",240,23,5.0,C,8888889.0
524,Ziaire Williams,76,"6'9""",185,22,4.0,SF,6133005.0


In [49]:
nba['height_cm'] = nba['height_feet'].apply(feet_inches_to_cm)
nba['weight_kg'] = nba['weight_lbs'].apply(lbs_to_kg)
nba['APY']=nba['season_salary']
nba.drop(['height_feet', 'weight_lbs','season_salary'], axis=1, inplace=True)
nba.dropna(subset=['APY'], inplace=True)
nba.rename(columns={'Full Name': 'name', 'years_in_the_nba': 'years_pro', 'position_1': 'position'}, inplace=True)
nba= nba[['name', 'overall', 'height_cm', 'weight_kg', 'Age', 'years_pro', 'position', 'APY']]
nba

<ipython-input-49-78ea1a2fa504>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['height_cm'] = nba['height_feet'].apply(feet_inches_to_cm)
<ipython-input-49-78ea1a2fa504>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['weight_kg'] = nba['weight_lbs'].apply(lbs_to_kg)
<ipython-input-49-78ea1a2fa504>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,name,overall,height_cm,weight_kg,Age,years_pro,position,APY
0,Aaron Gordon,85,203,107,28,11.0,PF,22841455.0
1,Aaron Holiday,74,183,84,27,7.0,PG,4668000.0
2,Aaron Nesmith,79,196,98,24,5.0,SF,11000000.0
3,Aaron Wiggins,79,196,86,25,4.0,PF,10514017.0
7,A.J. Green,77,193,86,24,3.0,SG,2120693.0
...,...,...,...,...,...,...,...,...
521,Zach Edey,79,226,138,22,1.0,C,5756880.0
522,Zach LaVine,85,196,91,29,11.0,SG,43031940.0
523,Zeke Nnaji,72,206,109,23,5.0,C,8888889.0
524,Ziaire Williams,76,206,84,22,4.0,SF,6133005.0


In [59]:
fifa = fifa_raw[fifa_raw['fifa_version'] == 24][['long_name', 'overall', 'height_cm','weight_kg', 'age', 'league_level','player_positions', 'wage_eur']]
fifa.drop_duplicates('long_name', keep='first', inplace=True)
fifa = fifa[fifa['league_level'] == 1]
fifa['Age'] = fifa['age']
fifa.drop('age', axis=1, inplace=True)
fifa

,long_name,overall,height_cm,weight_kg,league_level,player_positions,wage_eur,Age
161671,Rúben Miguel Marques dos Santos Fernandes,71,187,81,1.0,CB,5000.0,37
161673,Cristiano Ronaldo dos Santos Aveiro,86,187,85,1.0,ST,66000.0,38
161674,Karl Tommy Andreas Johansson,65,184,78,1.0,CB,850.0,41
161675,José Manuel Reina Páez,77,188,92,1.0,GK,14000.0,40
161676,Roque Luis Santa Cruz Cantero,70,191,93,1.0,ST,500.0,41
...,...,...,...,...,...,...,...,...
180015,Umut Yaşar Keçeci,58,182,73,1.0,ST,2000.0,19
180016,Rakan Al Kaabi,54,174,71,1.0,"CDM, CM",2000.0,20
180018,Rodrigo Frutos,58,188,87,1.0,GK,500.0,20
180019,Christian Bos,55,180,70,1.0,RB,500.0,18


In [60]:
def wage_to_salary(wage_eur):
  return wage_eur*52

In [61]:
fifa['APY'] = fifa['wage_eur'].apply(wage_to_salary)
fifa.drop('wage_eur', axis=1, inplace=True)
fifa.rename(columns={'long_name': 'name',  'player_positions': 'position'}, inplace=True)
fifa= fifa[['name', 'overall', 'height_cm', 'weight_kg', 'Age', 'position', 'APY']]
fifa

,name,overall,height_cm,weight_kg,Age,position,APY
161671,Rúben Miguel Marques dos Santos Fernandes,71,187,81,37,CB,260000.0
161673,Cristiano Ronaldo dos Santos Aveiro,86,187,85,38,ST,3432000.0
161674,Karl Tommy Andreas Johansson,65,184,78,41,CB,44200.0
161675,José Manuel Reina Páez,77,188,92,40,GK,728000.0
161676,Roque Luis Santa Cruz Cantero,70,191,93,41,ST,26000.0
...,...,...,...,...,...,...,...
180015,Umut Yaşar Keçeci,58,182,73,19,ST,104000.0
180016,Rakan Al Kaabi,54,174,71,20,"CDM, CM",104000.0
180018,Rodrigo Frutos,58,188,87,20,GK,26000.0
180019,Christian Bos,55,180,70,18,RB,26000.0
